In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv('../data/sms_banking.csv', header=None)
df.columns = ['sms']
df

,sms
0,Nhap ma OTP 6190 de xac nhan chuyen 6.170.000 ...
1,"TK ViettelPay 9704...0765 GD -525,000 VND luc ..."
2,"TK ViettelPay 9704...0765 GD +1,232,043 VND lu..."
3,"SD TK 0011004331555 +49,990,000VND luc 20-08-2..."
4,"Ma OTP xac thuc GD la 389235, hieu luc 1 phut...."
5,"SD TK 0011004331555 -864,439VND luc 20-08-2022..."
6,"Thu phi thuong nien the Visa Debit, tu thang 0..."


In [3]:
for mes in df.sms:
    print(mes)
    print()

Nhap ma OTP 6190 de xac nhan chuyen 6.170.000 VND cho NGUYEN THI BICH THU,so tai khoan 0560109234007, phi GD 0 VND. Quy khach KHONG CUNG CAP OTP cho bat ky ai.

TK ViettelPay 9704...0765 GD -525,000 VND luc 2021-12-28 19:57:48 So du 1,127,637 VND Phi: -0 VND ND: GD chuyen tien 211228799060765 016537 ck

TK ViettelPay 9704...0765 GD +1,232,043 VND luc 2021-12-27 13:31:26 So du 1,652,637 VND  ND: GD nhan tien 195805

SD TK 0011004331555 +49,990,000VND luc 20-08-2022 11:02:01. SD 50,568,970VND. Ref 020097042308201102002022d7d1985460.79636.110201.CHUYEN KHOAN

Ma OTP xac thuc GD la 389235, hieu luc 1 phut. Chi tiet GD: Chuyen khoan nhanh qua so TK, so tien 864,439 VND tren kenh Mobile cua dich vu VCB Digibank.

SD TK 0011004331555 -864,439VND luc 20-08-2022 19:39:54. SD 49,124,531VND. Ref MBVCB.2361079139.085523. Lan chuyen tien.CT tu 001100433155...

Thu phi thuong nien the Visa Debit, tu thang 06/2022 den thang 06/2022, No 428310...3712. So tien 5000 VND. So du tai thoi diem gd 49,119,53

In [4]:
df.sms = df.sms.str.replace('[,.]', '', regex=True)
df

,sms
0,Nhap ma OTP 6190 de xac nhan chuyen 6170000 VN...
1,TK ViettelPay 97040765 GD -525000 VND luc 2021...
2,TK ViettelPay 97040765 GD +1232043 VND luc 202...
3,SD TK 0011004331555 +49990000VND luc 20-08-202...
4,Ma OTP xac thuc GD la 389235 hieu luc 1 phut C...
5,SD TK 0011004331555 -864439VND luc 20-08-2022 ...
6,Thu phi thuong nien the Visa Debit tu thang 06...


In [5]:
sms_extract = pd.DataFrame(columns=['amount', 'date', 'balance', 'type'])
sms_extract

,amount,date,balance,type


In [6]:
sms_extract.amount = df.sms.str.extractall(r'[+-](\d+).?VND').astype(int)
sms_extract.type = df.sms.str.extractall(r'([+-])\d+.?VND')
sms_extract

amount date balance type
  match                            
1 0        525000  NaN     NaN    -
  1             0  NaN     NaN    -
2 0       1232043  NaN     NaN    +
3 0      49990000  NaN     NaN    +
5 0        864439  NaN     NaN    -

In [7]:
sms_extract.type = sms_extract.type.apply(lambda x: 'debit' if x == '-' else 'credit')
sms_extract

amount date balance    type
  match                               
1 0        525000  NaN     NaN   debit
  1             0  NaN     NaN   debit
2 0       1232043  NaN     NaN  credit
3 0      49990000  NaN     NaN  credit
5 0        864439  NaN     NaN   debit

In [8]:
sms_extract.date = df.sms.str.extractall(r'(\d{2}-\d{2}-\d{4}|\d{4}-\d{2}-\d{2})')
sms_extract

amount        date balance    type
  match                                      
1 0        525000  2021-12-28     NaN   debit
  1             0         NaN     NaN   debit
2 0       1232043  2021-12-27     NaN  credit
3 0      49990000  20-08-2022     NaN  credit
5 0        864439  20-08-2022     NaN   debit

In [9]:
sms_extract.balance = df.sms.str.extractall(r'(?:SD|So du) (\d+).?VND').astype(int)
sms_extract

amount        date     balance    type
  match                                          
1 0        525000  2021-12-28   1127637.0   debit
  1             0         NaN         NaN   debit
2 0       1232043  2021-12-27   1652637.0  credit
3 0      49990000  20-08-2022  50568970.0  credit
5 0        864439  20-08-2022  49124531.0   debit

In [14]:
def convert_to_date(input_date):
    if input_date:
        # Check form of input date
        print(input_date)
        if re.findall(r'\d{2}-\d{2}-\d{4}', input_date):
            return pd.to_datetime(input_date, '%d-%m-%Y')
        elif re.findall(r'\d{4}-\d{2}-\d{2}', input_date):
            return pd.to_datetime(input_date, '%Y-%m-%d')

In [15]:
sms_extract.date = sms_extract.date.apply(convert_to_date)
sms_extract

2021-12-28


AssertionError: 

In [ ]:
sms_extract.dropna(inplace=True)
sms_extract

,,amount,date,balance,type
,match,,,,
1,0,525000,2021-12-28,1127637.0,debit
2,0,1232043,2021-12-27,1652637.0,credit
3,0,49990000,20-08-2022,50568970.0,credit
5,0,864439,20-08-2022,49124531.0,debit
